In [1]:
#import necessary modules
import csv
import requests
import numpy as np
import pandas as pd

In [2]:
# Load the CSV file
df = pd.read_csv('data/Toronto Neighbourhood Data.csv')
df.drop(columns = ['Unnamed: 0'], axis = 1, inplace = True)

df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


#### Extract the neighborhood that are located near toronto only or containes the word Toronto in it


In [3]:
toronto_df = df[df['Borough'].str.contains('Toronto')].reset_index()

# drop the index column
toronto_df.drop(['index'], axis = 1, inplace = True)
toronto_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [4]:
from geopy.geocoders import Nominatim

# get the coordinates of the Toronto region
address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)

latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [5]:
# Visualize all the neighbourhoods on the Map
import folium

toronto_map = folium.Map(location= [latitude, longitude], zoom_start = 11)

# plot the points on the map
for lat, lng in zip(toronto_df['Latitude'], toronto_df['Longitude']):
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        color = 'yellow',
        fill= True, 
        fill_color = 'blue',
        fill_opacity = 0.7
    ).add_to(toronto_map)

toronto_map

In [6]:
# Visualize all the neighbourhoods on the Map
import folium

toronto_map = folium.Map(location = [latitude, longitude], zoom_start = 11)

# plot the points on the map
for lat, lng in zip(toronto_df['Latitude'], toronto_df['Longitude']):
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        color = 'yellow',
        fill= True, 
        fill_color = 'blue',
        fill_opacity = 0.7
    ).add_to(toronto_map)

toronto_map

# Add popup labels
latitudes, longitudes = toronto_df.Latitude, toronto_df.Longitude
names = toronto_df.Neighbourhood
for lat, lng, label in zip(latitudes, longitudes, names):
    folium.Marker(
        [lat, lng],
        popup = label
    ).add_to(toronto_map)

toronto_map

### Exploring the neighborhoods

In [7]:
# Define Foursquare Credentials

CLIENT_ID = 'OECZ01BUCTDUDGR0YUA4VKHV1T4COOU5535O5OKPCJFEAU3F' # your Foursquare ID
CLIENT_SECRET = 'FETLWMYN0AT51PPEKTIYBFRENECKX1OBTUMNN5U2BHI3TH0K' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OECZ01BUCTDUDGR0YUA4VKHV1T4COOU5535O5OKPCJFEAU3F
CLIENT_SECRET:FETLWMYN0AT51PPEKTIYBFRENECKX1OBTUMNN5U2BHI3TH0K


In [8]:
# Let's explore the first neighborhood

neighbor = toronto_df['Neighbourhood'].iloc[0]
neighbor

'Regent Park, Harbourfront'

#### Get the neighbourhood Co-Ordinates

In [9]:
neighbor_lat = toronto_df['Latitude'].iloc[0]
neighbor_lng = toronto_df['Longitude'].iloc[0]

neighbor_lat, neighbor_lng

(43.6542599, -79.3606359)

#### Get the top 100 venues that are in the radius of 500 meters of this neighbourhood

In [10]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbor_lat, 
    neighbor_lng, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=OECZ01BUCTDUDGR0YUA4VKHV1T4COOU5535O5OKPCJFEAU3F&client_secret=FETLWMYN0AT51PPEKTIYBFRENECKX1OBTUMNN5U2BHI3TH0K&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [11]:
# Get the response
import requests
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


results = requests.get(url).json()
results.keys()

dict_keys(['meta', 'response'])

In [12]:
# function to extract the category of the venue
def get_category_type(row):
    categories_list = row['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
# filter the results

venues = json_normalize(results['response']['groups'][0]['items'])

# Filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
venues = venues.loc[:, filtered_columns]

# filter the category of each result
venues['venue.categories'] = venues.apply(get_category_type, axis = 1)

# clean columns
venues.columns = [col.split(".")[-1] for col in venues.columns]

print('{} venues were returned by Foursquare'.format(venues.shape[0]))

venues.head()

44 venues were returned by Foursquare


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


### Explore the Neighbourhood in Toronto Region 

In [14]:
# function to return all the nearby venues of the Toronto Neighbourhood
def getNearbyVenues(names, latitudes, longitudes, radius = 500, LIMIT = 100):
    
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        # Get the response
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['categories'][0]['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng']
            ) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        
    nearby_venues.columns = ['Neighbourhood', 
        'Neighbourhood Latitude',
        'Neighbourhood Longitude',
        'Venue',
        'Venue Category',
        'Venue Latitude',
        'Venue Longitude']

    return(nearby_venues)
    

In [15]:
toronto_venues = getNearbyVenues(
    names = toronto_df['Neighbourhood'],
    latitudes = toronto_df['Latitude'],
    longitudes = toronto_df['Longitude']
)


In [16]:
print(toronto_venues.shape)
toronto_venues.head()

(1624, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category,Venue Latitude,Venue Longitude
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,Bakery,43.653447,-79.362017
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,Spa,43.654735,-79.359874
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,Restaurant,43.656369,-79.356980


#### Group All the venues by the neighborhoods

In [17]:
toronto_venues.groupby('Neighbourhood').count().head()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category,Venue Latitude,Venue Longitude
Neighbourhood,,,,,,
Berczy Park,55,55,55,55,55,55
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,68,68,68,68,68,68


In [18]:
# Total Number of Unique Categories
print('There are total of {} unique categories'.format(len(toronto_venues['Venue Category'].unique())))

There are total of 237 unique categories


### Analyze Each Neighbourhood

In [19]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix = "", prefix_sep = "")

# Add the Neighbourhood column
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood']

# Move the Neighbourhood column to the first columns
fixed_col = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_col]

toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [20]:
toronto_grpd = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grpd.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.018182,0.0,0.0,0.000000,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
2,"Business reply mail Processing Centre, South C...",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0625,0.0625,0.0625,0.125,0.125,0.0625,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
4,Central Bay Street,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.014706,0.0,0.0,0.014706,0.0,0.014706


In [21]:
toronto_grpd.shape

(39, 238)

#### Let's print N most common venues of some neighbourhood

In [22]:
def n_most_common(n, neighbourhood):
    temp = toronto_grpd[toronto_grpd['Neighbourhood'] == neighbourhood]
    temp = temp.T.reset_index()
    temp.columns = ['venues', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print("----" + neighbourhood + "----")
    print(temp.sort_values('freq', ascending = False).reset_index(drop = True).head(5))
    print('\n')

In [23]:
n_most_common(5, 'Berczy Park')
n_most_common(5, 'Central Bay Street')

----Berczy Park----
           venues  freq
0     Coffee Shop  0.09
1    Cocktail Bar  0.04
2        Beer Bar  0.04
3  Farmers Market  0.04
4     Cheese Shop  0.04


----Central Bay Street----
               venues  freq
0         Coffee Shop  0.18
1                Café  0.06
2      Sandwich Place  0.04
3  Italian Restaurant  0.04
4     Thai Restaurant  0.03




#### Creating a Data Frame with the 10 most common venues of Neighbourhood

In [24]:
def most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:].sort_values(ascending = False)
    
    return row_categories.index.values[0:num_top_venues]

In [25]:
indicators = ['st', 'nd', 'rd']
num_top_venues = 10

columns = ['Neighbourhood']

for ind in range(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind + 1))
    
neighbourhoods_venues = pd.DataFrame(columns = columns)
neighbourhoods_venues['Neighbourhood'] = toronto_grpd['Neighbourhood']

for ind in range(toronto_grpd.shape[0]):
    neighbourhoods_venues.iloc[ind, 1:] = most_common_venues(toronto_grpd.iloc[ind, 1:], num_top_venues)

neighbourhoods_venues.head()


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Farmers Market,Bakery,Beer Bar,Restaurant,Cheese Shop,Seafood Restaurant,Cocktail Bar,Concert Hall,Beach
1,"Brockton, Parkdale Village, Exhibition Place",Café,Nightclub,Breakfast Spot,Coffee Shop,Convenience Store,Furniture / Home Store,Bar,Italian Restaurant,Bakery,Climbing Gym
2,"Business reply mail Processing Centre, South C...",Gym / Fitness Center,Auto Workshop,Comic Shop,Park,Pizza Place,Recording Studio,Restaurant,Butcher,Burrito Place,Brewery
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport,Boat or Ferry,Bar,Rental Car Location,Sculpture Garden,Plane,Coffee Shop,Boutique
4,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Salad Place,Department Store,Thai Restaurant,Burger Joint,Bubble Tea Shop,Japanese Restaurant


### Cluster Neighbourhoods

In [26]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


# number of Clusters
kluster = 5
toronto_grpd_clustered = toronto_grpd.drop('Neighbourhood', 1)

# K-Means Clustering
kmeans = KMeans(n_clusters = kluster, random_state = 0).fit(toronto_grpd_clustered)

# Get the labels
kmeans.labels_[:10]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [27]:
neighbourhoods_venues.insert(0, 'Cluster Labels', kmeans.labels_)

In [28]:
toronto_merged = toronto_df

# Merge the toronto data and the venues data to get the Co-Ordinates also
toronto_merged = toronto_merged.join(neighbourhoods_venues.set_index('Neighbourhood'), on = 'Neighbourhood')

toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Yoga Studio,Event Space,Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Yoga Studio,Diner,Beer Bar,Smoothie Shop,Italian Restaurant,Sandwich Place,Restaurant,Café,Distribution Center
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Hotel,Bookstore,Pizza Place,Middle Eastern Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Restaurant,Cocktail Bar,Gastropub,Beer Bar,American Restaurant,Gym,Hotel,Clothing Store
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Pub,Health Food Store,Trail,Neighborhood,Yoga Studio,Dog Run,Diner,Discount Store,Distribution Center,Donut Shop


#### Visualize the resulting Cluster

In [29]:
cluster_map = folium.Map(location = [latitude, longitude], zoom_start = 11)

x = np.arange(kluster)
ys = [i + x + (i*x)**2 for i in range(kluster)]

colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))

rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lng, hood, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = str(hood) + ', Cluster ' + str(cluster)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = rainbow[cluster - 1],
        fill = True,
        fill_color = 'yellow',
        fill_opacity = 0.7
    ).add_to(cluster_map)


cluster_map

### Neighbourhood Segmentation

#### Cluster 1

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,Park,Bus Line,Swim School,Diner,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
21,"Forest Hill North & West, Forest Hill Road Park",Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Dim Sum Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
33,Rosedale,Park,Playground,Trail,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


#### Cluster 2

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,"Moore Park, Summerhill East",Playground,Trail,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


#### Cluster 3

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Yoga Studio,Event Space,Restaurant
1,"Queen's Park, Ontario Provincial Government",Coffee Shop,Yoga Studio,Diner,Beer Bar,Smoothie Shop,Italian Restaurant,Sandwich Place,Restaurant,Café,Distribution Center
2,"Garden District, Ryerson",Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Hotel,Bookstore,Pizza Place,Middle Eastern Restaurant
3,St. James Town,Coffee Shop,Café,Restaurant,Cocktail Bar,Gastropub,Beer Bar,American Restaurant,Gym,Hotel,Clothing Store
5,Berczy Park,Coffee Shop,Farmers Market,Bakery,Beer Bar,Restaurant,Cheese Shop,Seafood Restaurant,Cocktail Bar,Concert Hall,Beach
6,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Salad Place,Department Store,Thai Restaurant,Burger Joint,Bubble Tea Shop,Japanese Restaurant
7,Christie,Grocery Store,Café,Park,Coffee Shop,Candy Store,Baby Store,Italian Restaurant,Athletics & Sports,Restaurant,Nightclub
8,"Richmond, Adelaide, King",Coffee Shop,Café,Hotel,Bar,Restaurant,Gym,Clothing Store,Thai Restaurant,Office,Concert Hall
9,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Music Venue,Bank,Middle Eastern Restaurant,Bar,Café,Supermarket,Grocery Store,Park
10,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Hotel,Café,Fried Chicken Joint,Scenic Lookout,Brewery,Restaurant,Sporting Goods Shop,Music Venue


#### Cluster 4

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,The Beaches,Pub,Health Food Store,Trail,Neighborhood,Yoga Studio,Dog Run,Diner,Discount Store,Distribution Center,Donut Shop


#### Cluster 5

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,Music Venue,Garden,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
